In [1]:
import pandas as pd
import numpy as np
import nltk 
import re

In [2]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

In [51]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ai_ds_a1/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anantha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
data = pd.read_csv("train.csv", encoding='unicode_escape')
df = pd.read_csv("test.csv", encoding='unicode_escape')

In [4]:
df

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
...,...,...,...,...,...,...,...,...,...
4810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data['text'][27476]

' wish we could come see u on Denver  husband lost his job and can`t afford it'

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            3534 non-null   object 
 1   text              3534 non-null   object 
 2   sentiment         3534 non-null   object 
 3   Time of Tweet     3534 non-null   object 
 4   Age of User       3534 non-null   object 
 5   Country           3534 non-null   object 
 6   Population -2020  3534 non-null   float64
 7   Land Area (Km²)   3534 non-null   float64
 8   Density (P/Km²)   3534 non-null   float64
dtypes: float64(3), object(6)
memory usage: 338.7+ KB


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [5]:
data.dropna(inplace=True)
df.dropna(inplace=True)

In [6]:
data.columns

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

In [7]:
data.drop(['textID', 'selected_text', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'], axis=1, inplace=True)

df.drop(['textID', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'], axis=1, inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27480 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27480 non-null  object
dtypes: object(2)
memory usage: 644.1+ KB


In [9]:
df

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
3529,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,All alone in this old house again. Thanks for...,positive
3531,I know what you mean. My little dog is sinkin...,negative
3532,_sutra what is your next youtube video gonna b...,positive


In [10]:
ps = PorterStemmer()

In [11]:
ps.stem("programmers")

'programm'

In [12]:
stop_words = set(stopwords.words('english'))

In [13]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text) 
    text = re.sub(r'\s+', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [ps.stem(word) for word in tokens]
    return ' '.join(tokens)

In [14]:
data['text'] = data['text'].apply(preprocess_text)
df['text'] = df['text'].apply(preprocess_text)

In [15]:
re.sub(r'\W', ' ','how old do i feel egh blah and boooooooooooo ã â â½ i dunno wanna go to work hangovers suckkkkkk im a drunk mess ')

'how old do i feel egh blah and boooooooooooo ã â â½ i dunno wanna go to work hangovers suckkkkkk im a drunk mess '

In [16]:
frames = [data, df]
 
dff = pd.concat(frames)

In [27]:
dff

,text,sentiment
0,respond go,1
1,sooo sad miss san diego,0
2,boss bulli,0
3,interview leav alon,0
4,son put releas alreadi bought,0
...,...,...
3529,3 im tire sleep tri,0
3530,alon old hous thank net keep aliv kick whoever...,2
3531,know mean littl dog sink depress want move som...,0
3532,_sutra next youtub video gonna love video,2


In [25]:
from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 
dff['sentiment'] = label_encoder.fit_transform(dff['sentiment']) 

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [30]:
sentences = dff['text'].tolist()
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]
sentence_string = [' '.join(tokens) for tokens in tokenized_sentences]

In [31]:
x_train_bow, x_test_bow, y_train, y_test = train_test_split(tokenized_sentences, dff['sentiment'], test_size=0.2, random_state=42)

In [32]:
vectorizer = CountVectorizer()
x_train_bow = vectorizer.fit_transform(x_train)
x_test_bow = vectorizer.fit_transform(x_test)
x_train_bow, x_test_bow, y_train, y_test = train_test_split(tokenized_sentences, dff['sentiment'], test_size=0.2, random_state=42)

In [34]:
clf_bow = RandomForestClassifier()
clf_bow.fit(x_train_bow, y_train)

RandomForestClassifier()

In [76]:
y_pred_bow = clf_bow.predict(x_test_bow)
accuracy_bow = accuracy_score(y_test, y_pred_bow)
print("Accuracy using Bag of Words:", accuracy_bow)

Accuracy using Bag of Words: 0.7017572142511688


In [40]:
vectorizer_tfidf = TfidfVectorizer()
x_train_tfidf = vectorizer_tfidf.fit_transform(x_train)
x_test_tfidf = vectorizer_tfidf.transform(x_test)

In [41]:
clf_bow = RandomForestClassifier()
clf_bow.fit(x_train_tfidf, y_train)

RandomForestClassifier()

In [42]:
y_pred_tfidf = clf_bow.predict(x_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print("Accuracy using TF IDF:", accuracy_tfidf)

Accuracy using TF IDF: 0.7046590359503466


In [43]:
word2vec_model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

In [44]:
def get_vector(tokens):
    sum(word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv)

In [46]:
x_train_w2v = [get_vector(tokens) for tokens in tokenized_sentences]
x_test_w2v = [get_vector(word_tokenize(sentence.lower())) for sentence in x_test]

In [49]:
clf_w2v = RandomForestClassifier()
clf_w2v.fit(x_train_w2v, y_train)

In [50]:
y_pred_w2v = clf_bow.predict(x_test_w2v)
accuracy_w2v = accuracy_score(y_test, y_pred_w2v)
print("Accuracy using Word2Vec:", accuracy_w2v)